In [1]:
from diffusers import StableDiffusionPipeline, StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionControlNetPipeline
from diffusers.loaders import AttnProcsLayers
from controlnet_aux import CannyDetector
import torch, PIL.Image as Image
import gc

c:\Users\artem\anaconda3\envs\generative-ai\Lib\site-packages\controlnet_aux\mediapipe_face\mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [2]:
controlnet = ControlNetModel.from_pretrained(
    'lllyasviel/sd-controlnet-canny', torch_dtype=torch.float16
)
canny = CannyDetector()
image = Image.open('dragon.jpg').convert('RGB')
control_image = canny(image)
control_image.save('canny_output.png')


In [3]:
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5', controlnet=controlnet, torch_dtype=torch.float16
).to('cuda')
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# LoRA адаптація
pipe.load_lora_weights('BJEon121/trained-sd1.5_fancy_boot', weights_name='pytorch_lora_weights.safetensors')



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [4]:
prompt = 'a majestic dragon sitting on the mountain, watercolor style'
image = pipe(prompt, image=control_image, guidance_scale=9, num_inference_steps=30).images[0]
image.save('final_result.png')

del pipe
torch.cuda.empty_cache()

  0%|          | 0/30 [00:00<?, ?it/s]

# Порівняння

In [5]:
prompt = 'a majestic dragon sitting on the mountain, watercolor style'

canny = CannyDetector()
input_image = Image.open('dragon.jpg').convert('RGB')
control_image = canny(input_image)
control_image.save('canny_output.png')

In [6]:
# 1. Базовий пайплайн (тільки prompt)
pipe_base = StableDiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    torch_dtype=torch.float16
).to('cuda')
pipe_base.scheduler = UniPCMultistepScheduler.from_config(pipe_base.scheduler.config)

img_base = pipe_base(prompt, guidance_scale=9, num_inference_steps=30).images[0]
img_base.save('result_base_prompt.png')

del pipe_base
torch.cuda.empty_cache()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
# 2. Пайплайн з ControlNet
controlnet = ControlNetModel.from_pretrained(
    'lllyasviel/sd-controlnet-canny', torch_dtype=torch.float16
)
pipe_controlnet = StableDiffusionControlNetPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    controlnet=controlnet,
    torch_dtype=torch.float16
).to('cuda')
pipe_controlnet.scheduler = UniPCMultistepScheduler.from_config(pipe_controlnet.scheduler.config)

img_controlnet = pipe_controlnet(prompt, image=control_image, guidance_scale=9, num_inference_steps=30).images[0]
img_controlnet.save('result_prompt_controlnet.png')

del pipe_controlnet
torch.cuda.empty_cache()
gc.collect()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

82

In [8]:
# 3. Базовий пайплайн + LoRA
pipe_lora = StableDiffusionPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    torch_dtype=torch.float16
).to('cuda')
pipe_lora.scheduler = UniPCMultistepScheduler.from_config(pipe_lora.scheduler.config)
pipe_lora.load_lora_weights('BJEon121/trained-sd1.5_fancy_boot', weight_name='pytorch_lora_weights.safetensors')

img_lora = pipe_lora(prompt, guidance_scale=9, num_inference_steps=30).images[0]
img_lora.save('result_prompt_lora.png')

del pipe_lora
torch.cuda.empty_cache()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/30 [00:00<?, ?it/s]

In [9]:
# 4. Пайплайн з ControlNet + LoRA
pipe_full = StableDiffusionControlNetPipeline.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    controlnet=controlnet,
    torch_dtype=torch.float16
).to('cuda')
pipe_full.scheduler = UniPCMultistepScheduler.from_config(pipe_full.scheduler.config)
pipe_full.load_lora_weights('BJEon121/trained-sd1.5_fancy_boot', weight_name='pytorch_lora_weights.safetensors')

img_full = pipe_full(prompt, image=control_image, guidance_scale=9, num_inference_steps=30).images[0]
img_full.save('result_prompt_lora_controlnet.png')

del pipe_full
torch.cuda.empty_cache()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


  0%|          | 0/30 [00:00<?, ?it/s]

In [10]:
gc.collect()

82

**1. У чому полягає суть адаптації LoRA?**  

LoRA (Low-Rank Adaptation) — це техніка донавчання великих моделей, яка дозволяє адаптувати поведінку моделі, додаючи невелику кількість нових параметрів замість повного перенавчання всіх ваг. Суть полягає в тому, що LoRA вносить зміни лише у певні шари (зазвичай cross-attention), вставляючи матриці низького рангу, що суттєво зменшує обсяг необхідної пам'яті та часу на навчання. Завдяки цьому модель зберігає оригінальні знання, одночасно навчаючись новим завданням або стилям.

**2. Які типи ControlNet можна використовувати?**  

ControlNet підтримує різні типи умовних вхідних даних, що формують структуру або стиль майбутнього зображення. Найпоширеніші типи включають: Canny-контури (крайові карти), Depth maps (карти глибини), Pose estimation (скелетні пози людини), Scribbles (ескізи), Segmentation maps (семантична сегментація), Normal maps (орієнтація поверхні) та OpenPose (детальне розпізнавання тіла). Кожен тип ControlNet допомагає більш точно керувати тим, що і як буде генеруватися.

**3. Як працює cross-attention в контексті LoRA і ControlNet?**  
Cross-attention — це механізм, за допомогою якого генеративна модель під час створення зображення враховує умови, такі як текстовий опис або контрольне зображення. У випадку LoRA модифікуються саме cross-attention шари, щоб легше пристосувати модель до нових стилів або специфічних тематик без повного перенавчання. ControlNet також інтегрується через cross-attention, де додаються додаткові контрольні сигнали, що впливають на розподіл уваги моделі між текстовим описом і структурною підказкою.

**4. У чому переваги поєднання тексту і зображення як умов?**  
Поєднання текстових і візуальних умов дозволяє досягати одночасно креативності й точності: текст визначає зміст, атмосферу та художній стиль сцени, а зображення (наприклад, контур або поза) забезпечує фізичну структуру й композицію. Таке комбіноване управління дає змогу створювати реалістичні сцени, відповідні обом вимогам одночасно, що важко реалізувати за допомогою лише одного типу умови.

**5. Які існують обмеження або потенційні проблеми при комбінуванні LoRA та ControlNet?**  
Комбінація LoRA і ControlNet може призводити до кількох викликів: збільшене споживання пам’яті (оскільки обидва методи додають додаткові шари/параметри), можливий конфлікт між стилем (LoRA) і структурою (ControlNet), що ускладнює досягнення бажаного результату без тонкої настройки. Також важливо правильно балансувати ваги впливу, інакше або LoRA буде "перебивати" структуру ControlNet, або навпаки — ControlNet буде занадто обмежувати творчість LoRA.